EMPEZAR

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


Utiliza correctamente técnicas avanzadas de NLP (tokenización, stopwords, stemming/lemmatización, extracción de polaridad, etc.). 
Explica y justifica el uso de cada técnica y evalúa su impacto en el análisis de sentimientos.

In [2]:
# DESCOMENTAR LA SIGUIENTE LÍNEA PARA DESCARGAR LOS RECURSOS NECESARIOS LA PRIMERA VEZ
# nltk.download()
# Descargar recursos
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')

# Inicialización de herramientas
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
sia = SentimentIntensityAnalyzer()

def preprocesar_texto(texto):
    # Tokenización
    tokens = word_tokenize(texto.lower())
    
    # Eliminación de stopwords
    tokens_sin_stopwords = [word for word in tokens if word not in stop_words and word.isalpha()]
    
    # Lematización
    tokens_lemmatized = [lemmatizer.lemmatize(word) for word in tokens_sin_stopwords]
    
    return " ".join(tokens_lemmatized)

def analizar_sentimiento(texto, calificacion):
    # Análisis de sentimiento del texto con VADER
    puntajes = sia.polarity_scores(texto)
    sentimiento_texto = 'neutral'  # Valor predeterminado
    if puntajes['compound'] >= 0.05:
        sentimiento_texto = 'positiva'
    elif puntajes['compound'] <= -0.05:
        sentimiento_texto = 'negativa'

    # Análisis de sentimiento basado en la calificación
    if calificacion in [4, 5]:
        sentimiento_calificacion = 'positiva'
    elif calificacion == 3:
        sentimiento_calificacion = 'neutral'
    else:
        sentimiento_calificacion = 'negativa'

    # Combinar ambos análisis de sentimientos
    if sentimiento_texto == sentimiento_calificacion:
        return sentimiento_texto
    else:
        return sentimiento_texto  # Priorizar el sentimiento del texto

def procesar_datos(df):
    # Aplicar preprocesamiento y análisis de sentimientos a cada reseña
    df['processed_text'] = df['reviewText'].apply(preprocesar_texto)
    df['sentimientos'] = df.apply(lambda row: analizar_sentimiento(row['processed_text'], row['overall']), axis=1)
    # Transformar 'reviewTime' a un formato numérico (podemos usar solo el año para simplicidad)
    df['reviewYear'] = pd.to_datetime(df['reviewTime']).dt.year
    return df

# Ejemplo de uso con el DataFrame
df = pd.read_json("data/web_reviews.json")
df = procesar_datos(df)

# Vectorización del texto
vectorizer = TfidfVectorizer(max_features=1000)  # Convertir texto en representación numérica
X_text = vectorizer.fit_transform(df['processed_text']).toarray()

# Concatenar texto vectorizado con otras columnas numéricas
X_otro = df[['overall', 'reviewYear']].values

X = np.hstack([X_text, X_otro])  # Combinar texto y características numéricas

y = df['sentimientos']

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aneja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aneja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aneja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aneja\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\aneja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# MODELO 1 --> RandomForestClassifier 

# ESPECIFICAR QUE PASA SIN TOCAR LOS HIPERPARAMETROS Y TOCANDOLOS EDITANDO 

In [3]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar y evaluar RandomForestClassifier
modelo_RFC_sh = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_RFC_sh.fit(X_train, y_train)

# Predicciones y evaluación
y_pred_RFC_sh = modelo_RFC_sh.predict(X_test)

print("Accuracy del modelo:", accuracy_score(y_test, y_pred_RFC_sh))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred_RFC_sh))

# --------------------------CON HIPERPARAMETROS --------------------------------------------


# Definir el modelo base
modelo_RFC = RandomForestClassifier(random_state=42)

# Definir el rango de hiperparámetros para la búsqueda
param_grid_rfc = {
    'n_estimators': [50, 100, 200],               # Número de árboles
    'max_depth': [None, 10, 20, 30],               # Profundidad máxima del árbol
    'min_samples_split': [2, 5, 10]             # Mínimo número de muestras para dividir un nodo
}

# Usar GridSearchCV para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=modelo_RFC, param_grid=param_grid_rfc, cv=5, n_jobs=-1, verbose=2)

# Ajustar el modelo con la búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros
print(f"Mejores hiperparámetros: {grid_search.best_params_}")

# Entrenar el modelo con los mejores hiperparámetros encontrados
modelo_RFC_mejorado = grid_search.best_estimator_

# Predicciones y evaluación
y_pred_RFC = modelo_RFC_mejorado.predict(X_test)

print("Accuracy del modelo con los mejores hiperparámetros:", accuracy_score(y_test, y_pred_RFC))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred_RFC))

Accuracy del modelo: 0.8962493911349245

Reporte de clasificación:
               precision    recall  f1-score   support

    negativa       1.00      0.02      0.04       159
     neutral       0.20      0.02      0.03        54
    positiva       0.90      1.00      0.95      1840

    accuracy                           0.90      2053
   macro avg       0.70      0.35      0.34      2053
weighted avg       0.89      0.90      0.85      2053

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Mejores hiperparámetros: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}
Accuracy del modelo con los mejores hiperparámetros: 0.8972235752557234

Reporte de clasificación:
               precision    recall  f1-score   support

    negativa       1.00      0.02      0.04       159
     neutral       0.33      0.02      0.04        54
    positiva       0.90      1.00      0.95      1840

    accuracy                           0.90      2053
   macro avg       0.74    

# MODELO 2 --> SVC

In [4]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar y evaluar SVC
modelo_SVC_sh = SVC(kernel='linear', random_state=42)  # Usamos un kernel lineal
modelo_SVC_sh.fit(X_train, y_train)

# Predicciones y evaluación
y_pred_svc_sh = modelo_SVC_sh.predict(X_test)

print("Accuracy del modelo SVC:", accuracy_score(y_test, y_pred_svc_sh))
print("\nReporte de clasificación (SVC):\n", classification_report(y_test, y_pred_svc_sh))


# --------------------------CON HIPERPARAMETROS --------------------------------------------

# Definir el modelo base
modelo_SVC = SVC(random_state=42)

# Definir el rango de hiperparámetros para la búsqueda
param_grid_svc = {
    'C': [0.1, 1, 10],            # Penalización por error
    'kernel': ['linear', 'rbf', 'sigmoid'],  # Tipos de kernel
    'gamma': ['scale', 'auto'],  # Función de kernel para SVM no lineales
}

# Usar GridSearchCV para encontrar los mejores hiperparámetros
grid_search_svc = GridSearchCV(estimator=modelo_SVC, param_grid=param_grid_svc, cv=5, n_jobs=-1, verbose=2)

# Ajustar el modelo con la búsqueda de hiperparámetros
grid_search_svc.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros
print(f"Mejores hiperparámetros SVC: {grid_search_svc.best_params_}")

# Entrenar el modelo con los mejores hiperparámetros encontrados
modelo_SVC_mejorado = grid_search_svc.best_estimator_

# Predicciones y evaluación
y_pred_svc = modelo_SVC_mejorado.predict(X_test)

print("Accuracy del modelo SVC con mejores hiperparámetros:", accuracy_score(y_test, y_pred_svc))
print("\nReporte de clasificación (SVC):\n", classification_report(y_test, y_pred_svc))


Accuracy del modelo SVC: 0.8977106673161227

Reporte de clasificación (SVC):
               precision    recall  f1-score   support

    negativa       1.00      0.02      0.04       159
     neutral       0.00      0.00      0.00        54
    positiva       0.90      1.00      0.95      1840

    accuracy                           0.90      2053
   macro avg       0.63      0.34      0.33      2053
weighted avg       0.88      0.90      0.85      2053

Fitting 5 folds for each of 18 candidates, totalling 90 fits


d:\IA+BD\envs\5073\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\IA+BD\envs\5073\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\IA+BD\envs\5073\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mejores hiperparámetros SVC: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy del modelo SVC con mejores hiperparámetros: 0.8981977593765221

Reporte de clasificación (SVC):
               precision    recall  f1-score   support

    negativa       0.80      0.03      0.05       159
     neutral       0.00      0.00      0.00        54
    positiva       0.90      1.00      0.95      1840

    accuracy                           0.90      2053
   macro avg       0.57      0.34      0.33      2053
weighted avg       0.87      0.90      0.85      2053



d:\IA+BD\envs\5073\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\IA+BD\envs\5073\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\IA+BD\envs\5073\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# MODELO 3 --> LogisticRegression

In [5]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar y evaluar Logistic Regression
modelo_LR_sh = LogisticRegression(max_iter=1000, random_state=42)
modelo_LR_sh.fit(X_train, y_train)

# Predicciones y evaluación
y_pred_lr_sh = modelo_LR_sh.predict(X_test)

print("Accuracy del modelo Logistic Regression:", accuracy_score(y_test, y_pred_lr_sh))
print("\nReporte de clasificación (Logistic Regression):\n", classification_report(y_test, y_pred_lr_sh))

# --------------------------CON HIPERPARAMETROS --------------------------------------------


# Definir el modelo base
modelo_LR = LogisticRegression(max_iter=1000, random_state=42)

# Definir el rango de hiperparámetros para la búsqueda
param_grid_lr = {
    'C': [0.1, 1, 10]               # Parámetro de regularización
}

# Usar GridSearchCV para encontrar los mejores hiperparámetros
grid_search_lr = GridSearchCV(estimator=modelo_LR, param_grid=param_grid_lr, cv=5, n_jobs=-1, verbose=2)

# Ajustar el modelo con la búsqueda de hiperparámetros
grid_search_lr.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros
print(f"Mejores hiperparámetros Logistic Regression: {grid_search_lr.best_params_}")

# Entrenar el modelo con los mejores hiperparámetros encontrados
modelo_LR_mejorado = grid_search_lr.best_estimator_

# Predicciones y evaluación
y_pred_lr = modelo_LR_mejorado.predict(X_test)

print("Accuracy del modelo Logistic Regression con mejores hiperparámetros:", accuracy_score(y_test, y_pred_lr))
print("\nReporte de clasificación (Logistic Regression):\n", classification_report(y_test, y_pred_lr))



d:\IA+BD\envs\5073\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\IA+BD\envs\5073\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\IA+BD\envs\5073\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sa

Accuracy del modelo Logistic Regression: 0.8996590355577204

Reporte de clasificación (Logistic Regression):
               precision    recall  f1-score   support

    negativa       0.64      0.09      0.15       159
     neutral       0.00      0.00      0.00        54
    positiva       0.90      1.00      0.95      1840

    accuracy                           0.90      2053
   macro avg       0.51      0.36      0.37      2053
weighted avg       0.86      0.90      0.86      2053

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Mejores hiperparámetros Logistic Regression: {'C': 10}
Accuracy del modelo Logistic Regression con mejores hiperparámetros: 0.896736483195324

Reporte de clasificación (Logistic Regression):
               precision    recall  f1-score   support

    negativa       0.47      0.23      0.31       159
     neutral       0.37      0.13      0.19        54
    positiva       0.92      0.98      0.95      1840

    accuracy                           

d:\IA+BD\envs\5073\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Selecciona la mejor estrategia considerando el contexto del dato (imputación con media, mediana, moda, o eliminación). Justifica claramente la decisión.

In [ ]:
#Eliminación variables categoricas 
# Seleccionamos solo las columnas numéricas y eliminamos variables categóricas
X_numerico = df.select_dtypes(exclude=['object'])
X_train, X_test, y_train, y_test = train_test_split(X_numerico, y, test_size=0.2, random_state=1)

# Entrenamos un modelo de Random Forest usando solo variables numéricas
modelo_rf = RandomForestClassifier(random_state=1)
modelo_rf.fit(X_train, y_train)

# Evaluamos el modelo
accuracy = modelo_rf.score(X_test, y_test)
print(f'Precisión del modelo tras eliminar variables categóricas: {accuracy:.2f}')


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Seleccionamos las columnas categóricas
categoricas = df.select_dtypes(include=['object']).copy()

# Codificamos las columnas categóricas ordinales
ordinal_encoder = OrdinalEncoder()
categoricas_encoded = pd.DataFrame(ordinal_encoder.fit_transform(categoricas), columns=categoricas.columns)

# Combinamos las columnas numéricas y las codificadas ordinalmente
X_ordinal = pd.concat([X_numerico.reset_index(drop=True), categoricas_encoded], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_ordinal, y, test_size=0.2, random_state=1)

# Entrenamos el modelo
modelo_rf.fit(X_train, y_train)
accuracy = modelo_rf.score(X_test, y_test)
print(f'Precisión del modelo con codificación ordinal: {accuracy:.2f}')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report

def evaluar_modelos_con_estrategias(df, modelos, estrategias, columna_objetivo='sentimientos'):
    """
    Evalúa modelos con distintas estrategias de imputación o eliminación de valores faltantes.
    
    Parameters:
    - df: El DataFrame con los datos a evaluar.
    - modelos: Un diccionario con modelos preentrenados. Ejemplo {'modelo_LR': modelo_LR, 'modelo_SVC': modelo_SVC, ...}
    - estrategias: Una lista de las estrategias de imputación a probar. Ejemplo ['mean', 'median', 'most_frequent', 'eliminar'].
    - columna_objetivo: Nombre de la columna objetivo.

    Returns:
    - resultados: Diccionario con los resultados de cada modelo y cada estrategia.
    """
    resultados = {}

    # Filtrar solo columnas numéricas, excluyendo la columna objetivo
    columnas_numericas = df.select_dtypes(include=['number']).columns
    columnas_numericas = [col for col in columnas_numericas if col != columna_objetivo]

    # Iterar por cada estrategia
    for estrategia in estrategias:
        if estrategia == 'eliminar':
            # Si la estrategia es eliminar, eliminamos las filas con valores faltantes en la columna objetivo
            df_imputado = df.dropna(subset=[columna_objetivo] + list(columnas_numericas))
        else:
            # Imputar con la estrategia seleccionada (solo en las columnas numéricas)
            imputer = SimpleImputer(strategy=estrategia)
            df_imputado = df.copy()

            # Aplicamos la imputación a las columnas numéricas
            df_imputado[columnas_numericas] = imputer.fit_transform(df_imputado[columnas_numericas])

        # Separar las características (X) y la variable objetivo (y)
        X = df_imputado[columnas_numericas]  # Excluir columna objetivo
        y = df_imputado[columna_objetivo]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Evaluar cada modelo con los datos imputados
        for nombre_modelo, modelo in modelos.items():
            print(f"\nEvaluando modelo {nombre_modelo} con imputación {estrategia}...")
            
            # Ajustar el modelo con los datos de entrenamiento
            modelo.fit(X_train, y_train)
            
            # Realizar predicciones
            y_pred = modelo.predict(X_test)
            
            # Evaluar rendimiento
            accuracy = accuracy_score(y_test, y_pred)
            clasif_report = classification_report(y_test, y_pred)
            
            # Almacenar los resultados
            if nombre_modelo not in resultados:
                resultados[nombre_modelo] = {}
            if estrategia not in resultados[nombre_modelo]:
                resultados[nombre_modelo][estrategia] = {'accuracy': accuracy, 'classification_report': clasif_report}
            
            # Imprimir resultados
            print(f"Accuracy: {accuracy}")
            print(f"Reporte de clasificación:\n{clasif_report}")
    
    return resultados

# Ejemplo de cómo usar la función con tus modelos creados:

# Crear un diccionario con los modelos (asegúrate de tener modelos entrenados)
modelos = {
    'Logistic Regression': modelo_LR_mejorado,
    'SVC': modelo_SVC_mejorado,
    'Random Forest': modelo_RFC_mejorado
}

# Cargar el DataFrame (suponiendo que tienes un DataFrame 'df' con valores faltantes)
# df = pd.read_csv("ruta_del_archivo.csv")

# Definir las estrategias de imputación a probar
estrategias = ['mean', 'median', 'most_frequent', 'eliminar']

# Llamar a la función para evaluar los modelos con las diferentes estrategias de imputación
resultados = evaluar_modelos_con_estrategias(df, modelos, estrategias)

# Mostrar los resultados finales
print("\nResumen de resultados:")
for modelo, estrategias_resultados in resultados.items():
    print(f"\nModelo: {modelo}")
    for estrategia, resultado in estrategias_resultados.items():
        print(f"  Estrategia: {estrategia}")
        print(f"    Accuracy: {resultado['accuracy']}")
        print(f"    Reporte de clasificación:\n{resultado['classification_report']}")



Evaluando modelo Logistic Regression con imputación mean...
Accuracy: 0.8962493911349245
Reporte de clasificación:
              precision    recall  f1-score   support

    negativa       0.00      0.00      0.00       159
     neutral       0.00      0.00      0.00        54
    positiva       0.90      1.00      0.95      1840

    accuracy                           0.90      2053
   macro avg       0.30      0.33      0.32      2053
weighted avg       0.80      0.90      0.85      2053


Evaluando modelo SVC con imputación mean...


d:\IA+BD\envs\5073\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\IA+BD\envs\5073\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\IA+BD\envs\5073\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
